In [1]:
# import library
import pandas as pd

# Chapter 2. 고급 스크레핑

## 로그인이 필요한 사이트에서 다운받기
>### HTTP 통신
- 웹 브라우저와 웹 서버는 HTTP 통신규약(PROTOCOL)을 사용해서 통신
- 브라우저에서 서버로 요청(request)하면, 서버에서 브라우저로 응답(response)할 때 어떻게 할 것인지를 나타내는 규약
    - 웹 브라우저로 http://www.naver.com이라는 웹 서버 탐색
    - 웹 서버가 발견되면 index.html 파일을 보고싶다고 요청
    - naver.com 서버가 이러한 요청을 받으면 index.html 파일의 내용을 응답
- 같은 URL에 여러 번 접근해도 같은 데이터를 돌려주는 무상태(stateless) 통신

>### 쿠키
- 웹 브라우저를 통해 사이트에 방문하는 사람의 컴퓨터에 일시적으로 데이터를 저장하는 기능
- 1개의 쿠키에 저장할 수 있는 데이터의 크기는 4,096 byte로 제한
- HTTP 통신 헤더를 통해 읽고 쓰기가 가능
- 방문자 또는 확인자 측에서 원하는 대로 변경 가능
- 변경하면 문제가 될 비밀번호 등의 정보를 저장하기는 알맞지 않음

>### 세션
- 쿠키를 사용해 데이터를 저장
- 쿠키에는 방문자 고유 ID만 저장, 모든 데이터는 웹 서버에 저장
- 저장할 수 있는 데이터에 제한이 없음
- 회원제 웹 사이트 등의 구현이 가능

>### requests 사용
- `urllib.requests`를 이용해 쿠키를 이용한 접근이 가능
    - 방법이 조금 복잡
- requests 패키지를 사용하면 쉽게 쿠키를 이용한 접근이 가능
- 프로그램(봇 등)이 쉽게 로그인할 수 없게 보안 처리된 네이버 또는 다음 등 포털 사이트 등은 지금 사용하는 방법으로 로그인 불가능

>### 한빛출판네트워크
- site : http://www.hanbit.co.kr/member/login.html
- 입력양식(input)으로 m_id와 m_passwd라는 값(name 속성의 값)을 입력하여 입력양식을 제출하면 로그인되는 구조

>### 로그인 과정 분석
- 크롬 등에서 [검사] 화면을 띄우고 [Network] 탭을 띄워 어떠한 네트워크 통신이 오가는지 확인
- login_proc.php 선택
    - 로그인 관련 기능 처리
    - m_id와 m_passwd 정보 확인
    - http://www.hanbit.co.kr/member/login_proc.php 에 입력양식 데이터를 POST로 전달하면 로그인

>### 파이썬으로 로그인
- 마일리지와 이코인 출력

In [2]:
# 로그인을 위한 모듈 추출하기
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from getpass import getpass

# 아이디와 비밀번호 지정하기 [자신의 것을 사용해주세요]
USER = input('ID : ')
PASS = getpass('Password : ')

# 세션 시작하기
session = requests.session()

# 로그인하기
login_info = {
    'm_id' : USER,     # 아이디 지정
    'm_passwd' : PASS  # 비밀번호 지정
}
url_login = 'http://www.hanbit.co.kr/member/login_proc.php'
res = session.post(url_login, data=login_info)
res.raise_for_status() # 오류가 발생하면 예외 발생

ID : jinmang2
Password : ········


In [3]:
# 마이페이지 접근
url_mypage = 'http://www.hanbit.co.kr/myhanbit/myhanbit.html'
res = session.get(url_mypage)
res.raise_for_status()

# 마일리지와 이코인 가져오기
soup = BeautifulSoup(res.text, 'html.parser')
mileage = soup.select_one('.mileage_section1 span').get_text()
ecoin = soup.select_one('.mileage_section2 span').get_text()
print('마일리지 : {}'.format(mileage))
print('이코인 : {}'.format(ecoin))

마일리지 : 3,000
이코인 : 0


In [4]:
# 마이페이지 접근
url_mypage = 'http://www.hanbit.co.kr/myhanbit/membership.html'
res = session.get(url_mypage)
res.raise_for_status()

# 날짜별 순수구매금액과 적립마일리지 가져오기
soup = BeautifulSoup(res.text, 'html.parser')
date = soup.select_one('table.tbl_type_list2  tr  td').get_text()
buy = soup.select_one('table.tbl_type_list2 tr td.right').get_text()
month_mileage = soup.select_one('table.tbl_type_list2 tr td:nth-of-type(4)').get_text()

print('날짜 : {}'.format(date))
print('순수구매금액 : {}'.format(buy))
print('적립마일리지 : {}'.format(month_mileage))

날짜 : 2019 / 07
순수구매금액 : 0 원
적립마일리지 : 0 점


>### requests의 메서드
- HTTP에서 사용하는 `get`과 `post`등의 메서드는 `requests` 모듈에 같은 이름의 메서드가 존재
- `put`, `delete`, `head`와 같은 메서드도 존재

In [ ]:
# 로그인을 위한 모듈 추출하기
import requests

# get 요청
r = requests.get('http://google.com')
print(r.text, '\n')

# post 요청
formdata = {'key1' : 'value1', 'key2':'value2'}
r = requests.post('http://example.com', data=formdata)
print(r.content)

# 그 이외에 put, delete, head 등의 요청 메서드
r = requests.put('http://httpbin.org/put')
r = requests.delete('http://httpbin.org/delete')
r = requests.head('http://httpbin.org/get')

<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ko"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="8lVtfKnvBFplbNfZSKb/rg==">(function(){window.google={kEI:'CwQbXf-cGtjXhwPvzJ6wCw',kEXPI:'0,1353804,1958,2422,1225,730,224,510,1065,3152,56,322,206,904,113,175,110,777,60,81,281,240,385,2331452,329563,1294,12383,4855,32692,2074,13173,867,12163,5281,1953,9287,363,3320,5505,2436,266,5107,575,835,284,2,1306,2432,1361,4323,4968,773,2249,4745,2136,3577,3601,669,1050,1808,1397,81,7,491,2044,8909,5297,796,101,1119,38,622,298,746,8,119,1217,1364,346,1,1264,1580,1155,3062,2,632,3239,44,4148,635,1161,1446,632,2228,655,21,317,1119,447,457,187,2,1159,777,1,370,1315,486,29,190,756,98,392,30,399,992,98,1009,10,168,9,84,24,1018,1495,174,967,48,553,25,10,1269,2212,25,177,323,5,55,1190,7,327,513,324,193,531,371,361,221,38,70

b'<!doctype html>\n<html>\n<head>\n    <title>Example Domain</title>\n\n    <meta charset="utf-8" />\n    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1" />\n    <style type="text/css">\n    body {\n        background-color: #f0f0f2;\n        margin: 0;\n        padding: 0;\n        font-family: "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;\n        \n    }\n    div {\n        width: 600px;\n        margin: 5em auto;\n        padding: 50px;\n        background-color: #fff;\n        border-radius: 1em;\n    }\n    a:link, a:visited {\n        color: #38488f;\n        text-decoration: none;\n    }\n    @media (max-width: 700px) {\n        body {\n            background-color: #fff;\n        }\n        div {\n            width: auto;\n            margin: 0 auto;\n            border-radius: 0;\n            padding: 1em;\n        }\n    }\n    </style>    \n</head>\n\n<body>\n<div>\

>- 현재 시간에 대한 데이터를 추출하고 텍스트 형식과 바이너리 형식으로 출력
- 시간 모듈 공부 자료
    - https://godoftyping.wordpress.com/2015/04/19/python-%EB%82%A0%EC%A7%9C-%EC%8B%9C%EA%B0%84%EA%B4%80%EB%A0%A8-%EB%AA%A8%EB%93%88/
    - https://python.bakyeono.net/chapter-11-3.html

In [19]:
# 현재시간 데이터 가져오기
import requests
import time
import datetime
r = requests.get('http://api.aoikujira.com/time/get.php')

# 텍스트 형식으로 데이터 추출하기
text = r.text
print(text)

# 바이너리 형식으로 데이터 추출하기
binary = r.content
print(binary)

# time 모듈을 활용한 현재 시간 가져오기
print(time.gmtime(time.time()))

# datetime 모듈을 활용한 현재 시간 가져오기 
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

myDatetimeStr = '2015-04-15 12:23:38'
myDatetime = datetime.datetime.strptime(myDatetimeStr, '%Y-%m-%d %H:%M:%S')
print(type(myDatetime))
print(myDatetime)
 
yourDatetime = myDatetime.replace(day=16)
print(myDatetime)   # 2015-04-15 12:23:38
print(yourDatetime) # 2015-04-16 12:23:38

2019/07/02 16:24:42
b'2019/07/02 16:24:42'
time.struct_time(tm_year=2019, tm_mon=7, tm_mday=2, tm_hour=7, tm_min=24, tm_sec=42, tm_wday=1, tm_yday=183, tm_isdst=0)
2019-07-02 16:24:42
<class 'datetime.datetime'>
2015-04-15 12:23:38
2015-04-15 12:23:38
2015-04-16 12:23:38


## 웹 API로 데이터 추출하기
>### 웹 API(Application Programming Interface)
- 어떤 사이트가 가지고 있는 기능을 외부에서도 쉽게 사용할 수 있게 공개한 것
- 원래 어떤 프로그램 기능을 외부 프로그램에서 호출해서 사용할 수 있게 만든 것
    - 간단하게 서로 다른 프로그램이 기능을 공유하 수 있게 절차와 규약을 정의한 것
- 웹 API는 HTTP 통신을 사용하여 클라이언트 프로그램이 API를 제공하는 서버로 HTTP 요청을 보내면 서버가 이러한 요청을 기반으로 XML 또는 JSON 형식 등으로 응답
    - 클라이언트 $\rightarrow$ 서버 $\rightarrow$ 클라이언트
    - 각각 HTML 요청

>### OpenWeatherMap의 날씨 정보
- http://openweathermap.org
- 개발자 등록을 하고 API 키를 발급
- 유료 API
    - 현재 날씨, 5일까지의 날씨는 무료 사용
    - 1분에 600번까지 호출 가능
- 도시목록 데이터
    - http://bulk.openweathermap.org/sample/city.list.json.gz

In [24]:
import requests
import json
# API 키를 지정.
apikey = '0a02c9c74f92093dfa1b47666806574f'

# 날씨를 확인할 도시 지정하기
cities = ['seoul, KR', 'Tokyo, JP', 'New York, US']

# API 지정
api = 'http://api.openweathermap.org/data/2.5/weather?q={city}$APPID={key}'

# 켈빈 온도를 섭씨 온도로 변환하는 함수
k2c = lambda k : k - 273.15

# 각 도시의 정보 추출하기
for name in cities:
    # API의 URL 구성하기
    url = api.format(city=name, key=apikey)
    # API에 요청을 보내 데이터 추출하기
    r = requests.get(url)
    # 결과르 ㄹJSON 형식으로 변환하기
    data = json.loads(r.text)
    # 겨로가 출력하기 --- (%8)
    print('+ 도시 =', data['name'])
    print('| 날씨 =', data['weather'][0]['description'])
    print('| 최저 기온 =', k2c(data['main']['temp_min']))
    print('| 최고 기온 =', k2c(data['main']['temp_max']))
    print('| 습도 =', data['main']['humidity'])
    print('| 기압 =', data['main']['pressure'])
    print('| 풍향 =', data['wind']['deg'])
    print('| 풍속 =', data['wind']['speed'])
    print('')

KeyError: 'name'

>- JSON(JavaScript Object Notation)
    - '속성-값' 쌍 또는 '키-값' 쌍으로 이루어진 데이터 오브젝트를 전달하기 위해 인간이 읽을 수 있는 텍스트를 사용하는 개방형 표준 포멧
    - python의 dict와 거의 유사
- 국내 웹 API
    - API Store : http://www.apistore.co.kr/main.do
- 포탈 싸이트(네이버 개발자 센터와 다음 개발자 센터)
    - http://developers.naver.com/main/
    - http://developers.daum.net/
- 쇼핑 정보(옥션)
    - http://developer.auction.co.kr/
- 주소전환(행정자치부, 우체국)
    - http://www.juso.go.kr/openIndexPage.do
    - http://biz.epost.go.kr/ui/index.jsp

# Chapter 3. 데이터 소스의 서식과 가공

## 웹의 다양한 데이터 형식
>### 텍스트 데이터와 바이너리 데이터
- 텍스트 데이터는 일반적으로 텍스트 에디터로 편집할 수 있는 데이터 포맷
    - 자연어(한국어, 영어, 일본어 등) 및 숫자로 구성
    - 특수하게 줄 바꿈과 탭 등 제어 문자도 포함
    - XML, JSON, CSV 등
- 텍스트 데이터 이외의 데이터를 바이너리 데이터라고 부름
    - 바이너리는 문자와 상관없이 데이터를 사용할 수 있는 데이터 영역을 활용하는 데이터 형식
    - 텍스트 에디터로 열 수 없으며, 시각적으로 확인해도 의미를 알 수 없는 문자열로 표현
    - 텍스트 데이터보다 크기가 작음
        - 텍스트 3byte 파일을 바이너리 1btye로 저장
    - 동영상, 이미지 등은 대부분 바이너리 데이터

>### XML(extensible markup language) 분석
- XML은 텍스트 데이터를 기반으로 하는 형식
- 범용적인 형식으로 널리 사용
- 웹 API가 XML 형식을 활용
- 특정 목적에 따라서 태그로 감싸 마크업하는 범용적인 방식
- XML은 데이터를 계층 구조로 표현
    - 어떤 데이터 아래 서브 데이터를 추가 가능
- XML 기본 구조
        <요소 속성="값">내용</요소>
    - <요소> 태그로 감싸 마크업
    - 원한느 요소 이름을 사용
    - 하나의 요소에 속성을 사용해 여러 값을 추가로 지정
            <product id="S001" price="45000">SD 카드</product>
    - 다른 요소의 그룹으로 묶어 요소들이 계층 구조를 갖도록 작성
            <products type="전자제품>
                <product id="S001" price="45000">SD 카드</product>
                <product id="S002" price="32000">마우스</product>
            <products>

>### 파이썬으로 XML 분석하기
- BeautifulSoup을 이용하여 XML을 분석
- http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnld=108

In [26]:
from bs4 import BeautifulSoup
import urllib.request as req
import os.path

url = 'http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnld=108'
savename = 'forecast.xml'
if not os.path.exists(savename):
    req.urlretrieve(url, savename) # 로컬 파일로 저장

# BeautifulSoup으로 분석
with open(savename, 'r', encoding='utf-8') as f:
    xml = f.read()
    soup = BeautifulSoup(xml, 'html.parser') # html.parser 사용시 데이터의 태그를
                                             # 소문자로 처리
                                             # 따라서 소문자로 태그 입력
    # 각 지역 확인하기
    info = {}
    for location in soup.find_all('location'):
        name = location.find('city').string
        weather = location.find('wf').string
        if not (weather in info):
            info[weather] = []
        info[weather].append(name)
    
    # 각 지역의 날씨를 구분해서 출력하기
    for weather in info.keys():
        print('\n+', weather)
        for name in info[weather]:
            print('| -', name)


+ 맑음
| - 서울
| - 인천
| - 수원
| - 파주
| - 이천
| - 평택
| - 춘천
| - 원주
| - 강릉
| - 대전
| - 세종
| - 홍성
| - 청주
| - 충주
| - 영동
| - 광주
| - 목포
| - 여수
| - 순천
| - 광양
| - 나주
| - 전주
| - 군산
| - 정읍
| - 남원
| - 고창
| - 무주
| - 부산
| - 울산
| - 창원
| - 진주
| - 거창
| - 통영
| - 대구
| - 안동
| - 포항
| - 경주
| - 울진
| - 울릉도

+ 구름많음
| - 제주
| - 서귀포


>### JSON 분석
- JSON도 텍스트 데이터를 기반으로 하는 가벼운 데이터 형식
- JSON은 자바스크립트에서 사용하는 객체 표기 방법을 기반
- JSON은 자바스크립트 전용 데이터 형식은 아니며, 다양한 소프트웨어와 프로그래밍 언어끼리 데이터를 교환할 때 사용
- 확장자는 .json
- 파이썬 표준 모듈에도 json이 포함
- JSON의 구조
    - 숫자, 문자열, 논리, 배열, 객체, null의 6가지 종류의 데이터를 사용
    <table class="table table-striped table-bordered" style="width:600px">
        <thead>
            <tr>
                <th style="width:100px">자료형</th>
                <th style="width:250px">표현 방법</th>
                <th style="width:250px">사용 예</th>
            <tr>
        </thead>
        <tbody>
            <tr>
                <td>숫자</td>
                <td>숫자</td>
                <td>30</td>
            </tr>
            <tr>
                <td>문자열</td>
                <td>큰 따옴표로 감싸 표현</td>
                <td>"str"</td>
            </tr>
            <tr>
                <td>논리</td>
                <td>true 또는 false</td>
                <td>true, false</td>
            </tr>
            <tr>
                <td>배열</td>
                <td>[n1, n2, n3, ...]</td>
                <td>[1, 2, 10, 500]</td>
            </tr>
            <tr>
                <td>객체</td>
                <td>{"key1":value, "key2":value, ...}</td>
                <td>{"org":50, "com":10}</td>
            </tr>
            <tr>
                <td>null</td>
                <td>null</td>
                <td>null</td>
            </tr>
        </tbody>
    </table>
- 규칙은 단순하지만 배열 안에 객체를 넣거나 객체 안에 배열을 넣는 방법 등으로 복잡한 데이터를 표현
- JSON의 배열은 파이썬의 리스트, 객체는 파이썬의 dictionary와 동일
    - JSON 데이터 : https://api.github.com/repositories
    - 깃허브(github)는 Git을 사용하는 프로젝트를 지원하는 웹 호스팅 서비스로 오픈 소스 코드 저장소로 유명
    - 무작위로 레파지토리의 이름과 소유자를 추출해서 출력

In [27]:
import urllib.request as req
import os.path, random
import json

# JSON 데이터 내려받기
url = 'https://api.github.com/repositories'
savename = 'repo.json'
if not os.path.exists(savename):
    req.urlretrieve(url, savename)

# JSON 파일 분석하기
with open(savename, 'r', encoding='utf-8') as f:
#     items = json.load(f)
    json_data = f.read()
    items = json.loads(json_data)
    for item in items:
        print(item['name'] + " - " + item['owner']['login'])

grit - mojombo
merb-core - wycats
rubinius - rubinius
god - mojombo
jsawesome - vanpelt
jspec - wycats
exception_logger - defunkt
ambition - defunkt
restful-authentication - technoweenie
attachment_fu - technoweenie
microsis - caged
s3 - anotherjesse
taboo - anotherjesse
foxtracs - anotherjesse
fotomatic - anotherjesse
glowstick - mojombo
starling - defunkt
merb-more - wycats
thin - macournoyer
resource_controller - jamesgolick
markaby - jamesgolick
enum_field - jamesgolick
subtlety - defunkt
zippy - defunkt
cache_fu - defunkt
phosphor - KirinDave
sinatra - bmizerany
gsa-prototype - jnewland
duplikate - technoweenie
lazy_record - jnewland
gsa-feeds - jnewland
votigoto - jnewland
mofo - defunkt
xhtmlize - jnewland
ruby-git - ruby-git
bmhsearch - ezmobius
mofo - uggedal
simply_versioned - mmower
gchart - abhay
schemr - benburkert
calais - abhay
chronic - mojombo
git-wiki - sr
signal-wiki - queso
ruby-on-rails-tmbundle - drnic
low-pro-for-jquery - danwrong
merb-core - wayneeseguin
dst - s

>- JSON 형식으로 출력

In [29]:
# JSON 데이터 내려받기
price = {
    'date' : '2019-07-02',
    'price':{
        'Apple':80,
        'Orange':55,
        'Banana':40
    }}
s = json.dumps(price)
print(s)

{"date": "2019-07-02", "price": {"Apple": 80, "Orange": 55, "Banana": 40}}


>### 엑셀 파일 분석
- 파이썬에서 엑셀 파일을 읽고 쓸 때는 파이썬-엑셀 라이브러리를 사용
- openpyxl 패키지 설치

In [31]:
import openpyxl